<a href="https://colab.research.google.com/github/DOOPENGUIN/doksoo/blob/main/%EC%9D%B8%EC%A7%80%EA%B3%BC%ED%95%99_%EA%B3%BC%EC%9E%90%EB%AF%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install tensorflow keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import tensorflow as tf
import cv2 # 이미지 증강 관련 모듈
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import shutil
import random

In [4]:

def label_images(image_folders):
    image_extensions = (".jpg")  # 이미지 파일 확장자 목록

    labeled_images = []
    for folder_path, label in image_folders:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if any(file.lower().endswith(ext) for ext in image_extensions):
                    image_path = os.path.join(root, file)
                    labeled_images.append((image_path, label))

    return labeled_images

# 이미지 폴더 경로와 라벨 정보
image_folders = [
    ("/content/drive/MyDrive/이미지데이터들/철쭉/철쭉/철쭉/abnormal", "normal"),
    ("/content/drive/MyDrive/이미지데이터들/철쭉/철쭉/철쭉/normal", "abnormal")
]

# 이미지 파일 라벨링
labeled_images = label_images(image_folders)

# 라벨링된 이미지 출력
#for image_path, label in labeled_images:
#    print(f"Image Path: {image_path}  Label: {label}")

# 데이터 경로 설정
data_dir = '/content/drive/MyDrive/이미지데이터들/철쭉/철쭉/철쭉'

# train과 test 비율 설정
train_ratio = 0.7
test_ratio = 0.3

# abnormal 폴더 내부의 데이터 분할
abnormal_dir = os.path.join(data_dir, 'abnormal')
abnormal_files = os.listdir(abnormal_dir)
random.shuffle(abnormal_files)

abnormal_train_count = int(len(abnormal_files) * train_ratio)
abnormal_train_files = abnormal_files[:abnormal_train_count]
abnormal_test_files = abnormal_files[abnormal_train_count:]

train_dir = '/content/train'
test_dir = '/content/test'

# train 폴더에 파일 복사
os.makedirs(train_dir, exist_ok=True)
for file in abnormal_train_files:
    src_path = os.path.join(abnormal_dir, file)
    label = 'abnormal'
    dst_path = os.path.join(train_dir, f'{label}_{file}')
    shutil.copy(src_path, dst_path)

# test 폴더에 파일 복사
os.makedirs(test_dir, exist_ok=True)
for file in abnormal_test_files:
    src_path = os.path.join(abnormal_dir, file)
    label = 'abnormal'
    dst_path = os.path.join(test_dir, f'{label}_{file}')
    shutil.copy(src_path, dst_path)


# normal 폴더 내부의 데이터 분할
normal_dir = os.path.join(data_dir, 'normal')
normal_files = os.listdir(normal_dir)
random.shuffle(normal_files)

normal_train_count = int(len(normal_files) * train_ratio)
normal_train_files = normal_files[:normal_train_count]
normal_test_files = normal_files[normal_train_count:]

# train 폴더에 파일 복사
for file in normal_train_files:
    src_path = os.path.join(normal_dir, file)
    label = 'normal'
    dst_path = os.path.join(train_dir, f'{label}_{file}')
    shutil.copy(src_path, dst_path)

# test 폴더에 파일 복사
for file in normal_test_files:
    src_path = os.path.join(normal_dir, file)
    label = 'normal'
    dst_path = os.path.join(test_dir, f'{label}_{file}')
    shutil.copy(src_path, dst_path)

input_directory = "/content/train"  # argumentation을 수행할 이미지가 있는 디렉토리 경로
output_directory = "/content/augmentation"  # argumentation된 이미지를 저장할 디렉토리 경로

# 폴더 생성
os.makedirs(output_directory, exist_ok=True)

# 라벨링된 이미지를 순회하며 augmentation 수행
for image_filename in os.listdir(input_directory):
    image_path = os.path.join(input_directory, image_filename)
    img = cv2.imread(image_path)

    if img is None:  # 이미지를 로드할 수 없는 경우
        continue

    # 이미지를 원하는 augmentation 방식으로 변형
    augmented_images = []
    for i in range(3):  # 각 원본 이미지마다 3개의 augmentation 이미지 생성
        # 이미지 회전을 위한 augmentation 방식을 선택하고 적용
        angle = np.random.randint(-20, 20)  # -20도부터 20도 사이에서 무작위로 회전 각도 선택
        M = cv2.getRotationMatrix2D((img.shape[1] / 2, img.shape[0] / 2), angle, 1.0)
        augmented_img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
        augmented_images.append(augmented_img)

    # augmentation된 이미지를 저장
    for i, augmented_img in enumerate(augmented_images):
        filename = f"{os.path.splitext(image_filename)[0]}_augmented_{i}.jpg"
        output_path = os.path.join(output_directory, filename)
        cv2.imwrite(output_path, augmented_img)

In [13]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

# 데이터 경로 및 매개변수 설정
train_data_dir = '/content/augmentation'
test_data_dir = '/content/test'
image_size = (150, 150)
num_classes = 2  # "normal"과 "abnormal" 두 개의 클래스

# 결과 출력 경로 설정
output_dir = '/content/result_test'

# 데이터 로드 및 전처리
def load_and_preprocess_data(data_dir):
    images = []
    labels = []

    for filename in os.listdir(data_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img = load_img(os.path.join(data_dir, filename), target_size=image_size)
            img_array = img_to_array(img)
            images.append(img_array)
            label = filename.split("_")[0]  # 이미지 파일명에서 레이블 추출
            labels.append(label)

    images = np.array(images) / 255.0
    labels = np.array(labels)

    return images, labels

# 학습 데이터 로드 및 전처리
train_images, train_labels = load_and_preprocess_data(train_data_dir)
train_labels = np.where(train_labels == 'normal', 0, 1)  # "normal"을 0으로, "abnormal"을 1로 변환
train_labels = to_categorical(train_labels, num_classes)

# 테스트 데이터 로드 및 전처리
test_images, test_labels = load_and_preprocess_data(test_data_dir)
test_labels = np.where(test_labels == 'normal', 0, 1)  # "normal"을 0으로, "abnormal"을 1로 변환
test_labels = to_categorical(test_labels, num_classes)

# 신경망 모델 구성
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# 모델 컴파일 및 학습
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# ModelCheckpoint 콜백 설정
checkpoint_path = os.path.join(output_dir, 'model_checkpoint.h5')
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', save_weights_only=True, save_best_only=True)

# 학습
model.fit(train_images, train_labels, epochs=400, callbacks=[checkpoint], verbose=2)

# 테스트 데이터 평가
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

# 테스트 데이터에 대한 예측 수행
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)
predicted_labels = np.where(predicted_labels == 0, 'normal', 'abnormal')

# 예측 결과 이미지 저장
os.makedirs(output_dir, exist_ok=True)

for i in range(len(test_images)):
    img_filename = f"image_{i}_predicted_{predicted_labels[i]}.jpg"
    img_path = os.path.join(output_dir, img_filename)
    img_array = test_images[i] * 255.0
    img = img_array.astype(np.uint8)
    img = Image.fromarray(img)
    img.save(img_path)

print("Predictions saved to", output_dir)


Epoch 1/400


34/34 - 2s - loss: 0.8097 - accuracy: 0.6021 - 2s/epoch - 65ms/step
Epoch 2/400


34/34 - 0s - loss: 0.4945 - accuracy: 0.7837 - 442ms/epoch - 13ms/step
Epoch 3/400


34/34 - 0s - loss: 0.4215 - accuracy: 0.8137 - 453ms/epoch - 13ms/step
Epoch 4/400


34/34 - 0s - loss: 0.2645 - accuracy: 0.8904 - 442ms/epoch - 13ms/step
Epoch 5/400


...

34/34 - 0s - loss: 6.1391e-09 - accuracy: 1.0000 - 461ms/epoch - 14ms/step
Epoch 397/400


34/34 - 0s - loss: 6.9204e-09 - accuracy: 1.0000 - 456ms/epoch - 13ms/step
Epoch 398/400


34/34 - 0s - loss: 7.1436e-09 - accuracy: 1.0000 - 467ms/epoch - 14ms/step
Epoch 399/400


34/34 - 0s - loss: 5.6926e-09 - accuracy: 1.0000 - 455ms/epoch - 13ms/step
Epoch 400/400


34/34 - 0s - loss: 6.0274e-09 - accuracy: 1.0000 - 452ms/epoch - 13ms/step
5/5 [==============================] - 0s 8ms/step - loss: 4.6631 - accuracy: 0.6948
Test accuracy: 0.6948052048683167
5/5 [==============================] - 0s 7ms/step
Predictions saved to /content/result_test


In [14]:
# 폴더 경로 설정
input_folder = '/content/result_test'
output_folder = '/content/result'

# 입력 폴더에서 이미지 파일 목록을 가져옴
image_files = os.listdir(input_folder)

# 결과를 저장할 폴더 생성
os.makedirs(output_folder, exist_ok=True)

# 예측에 사용할 모델 정의
vgg_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=True)
grad_cam_model = tf.keras.models.Model([vgg_model.input],
                                       [vgg_model.get_layer('block5_conv3').output, vgg_model.output])

# 이미지 파일을 순회하며 Grad-CAM 수행
for image_file in image_files:
    # 이미지 경로 생성
    image_path = os.path.join(input_folder, image_file)

    # 이미지 불러오기
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Grad-CAM 수행
    conv_outputs, predictions = grad_cam_model.predict(np.reshape(rgb_img, (1, 224, 224, 3)))
    img_index = np.argmax(predictions)

    # Grad-CAM 결과 생성
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_cam_model(np.reshape(rgb_img, (1, 224, 224, 3)))
        loss = predictions[:, img_index]
    output = conv_outputs[0]
    grads = tape.gradient(loss, conv_outputs)[0]

    grad_weights = tf.reduce_mean(grads, axis=(0, 1))
    cam = np.ones(output.shape[0:2], dtype=np.float32)
    for i, w in enumerate(grad_weights):
        cam += w * output[:, :, i]

    cam = cv2.resize(cam.numpy(), (224, 224))
    grad_cam = (cam - cam.min()) / (cam.max() - cam.min())

    # 결과 이미지 저장 경로 생성
    output_path = os.path.join(output_folder, image_file)

    # 결과 이미지 저장
    cam = cv2.applyColorMap(np.uint8(255 * grad_cam), cv2.COLORMAP_JET)
    cam = cv2.cvtColor(cam, cv2.COLOR_RGB2BGR)
    result_img = cv2.add(cam, img)
    cv2.imwrite(output_path, result_img)

print("Grad-CAM 결과가 저장되었습니다.")


1/1 [==============================] - 0s 18ms/step
Grad-CAM 결과가 저장되었습니다.
